# Transformaciones de Datos

Usualmente no se tiene los datos en un formato conveniente. Una gran parte del trabajo con datos consiste en usar el conocimiento de un dominio determinado para saber cómo manejar los datos (eliminar algunos datos faltantes, realizar "feature engineering", transformar datos, etc.)

Spark tiene métodos para realizar estas transformaciones: http://spark.apache.org/docs/latest/ml-features.html

In [2]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## 1. Atributos categóricos a numéricos usando índices

Se puede utilizar `StringIndexer` para convertir atributos categóricos (no numéricos) en atributos numéricos.

In [4]:
from pyspark.ml.feature import StringIndexer

*Ejemplo*: creación manual de un dataframe donde el atributo "sucursal" es categórico (no es numérico)

In [54]:
from pyspark.sql import Row

df = spark.createDataFrame([Row(ID=0, sucursal="A", venta=10000), Row(ID=1, sucursal="B", venta=9000),
                            Row(ID=2, sucursal="C", venta=15000), Row(ID=3, sucursal="A", venta=14000),
                            Row(ID=4, sucursal="A", venta=12000), Row(ID=5, sucursal="C", venta=19000),
                            Row(ID=6, sucursal="D", venta=11500), Row(ID=7, sucursal="D", venta=5000)
])
df.show()

+---+--------+-----+
| ID|sucursal|venta|
+---+--------+-----+
|  0|       A|10000|
|  1|       B| 9000|
|  2|       C|15000|
|  3|       A|14000|
|  4|       A|12000|
|  5|       C|19000|
|  6|       D|11500|
|  7|       D| 5000|
+---+--------+-----+



Se convertirá la columna "sucursal" en numérica utilizando `StringIndexer`. Primero se indica cuál es la columna de entrada (`inputCol`) y cuál será la columna de salida (`indiceSucursal`)

In [55]:
# La columna con categoría indexada (numérica) se llamará "indiceCategoria"
indexador = StringIndexer(inputCol="sucursal", outputCol="indiceSucursal")

# Obtener las asociaciones entre categorías y valore numéricos (mapa)
modeloIndexador = indexador.fit(df)
# Mostrar las etiquetas que se mapean como (0, 1, 2)
modeloIndexador.labels

['A', 'C', 'D', 'B']

In [51]:
modeloIndexador

StringIndexerModel: uid=StringIndexer_3f7500b4fb00, handleInvalid=error

Luego se transforma los datos según los índices generados. Notar que se utiliza `transform` para realizar esta transformación

In [56]:
# Transformar los datos según los índices generados
df2 = modeloIndexador.transform(df)
df2.show()

+---+--------+-----+--------------+
| ID|sucursal|venta|indiceSucursal|
+---+--------+-----+--------------+
|  0|       A|10000|           0.0|
|  1|       B| 9000|           3.0|
|  2|       C|15000|           1.0|
|  3|       A|14000|           0.0|
|  4|       A|12000|           0.0|
|  5|       C|19000|           1.0|
|  6|       D|11500|           2.0|
|  7|       D| 5000|           2.0|
+---+--------+-----+--------------+



Luego de esta transformación, se puede utilizar el índice `indiceSucursal` como entrada numérica para algún algoritmo de Machine Learning.

## 2. Atributos categóricos a numéricos usando One-hot Encoding

La idea de one-hot encoding es mapear cada categoría a un vector binario con un solo valor que indique la presencia de un atributo particular (una característica específica).

In [27]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [57]:
# Data frame original
df.show()

+---+--------+-----+
| ID|sucursal|venta|
+---+--------+-----+
|  0|       A|10000|
|  1|       B| 9000|
|  2|       C|15000|
|  3|       A|14000|
|  4|       A|12000|
|  5|       C|19000|
|  6|       D|11500|
|  7|       D| 5000|
+---+--------+-----+



**Forma 1**: Manipulando directamente el indexador y el conversor a one-hot encoding

In [59]:
# Crear y aplicar un indexador
indexador = StringIndexer(inputCol="sucursal", outputCol="indiceSucursal")
# Transformar y aplicar el indexador al DataFrame
df2 = indexador.fit(df).transform(df)

# Definir One-Hot encoder
one_hot_encoder = OneHotEncoder(inputCol="indiceSucursal", outputCol="onehotSucursal")
# Transformar y aplicar OneHotEncoder al DataFrame
df3 = one_hot_encoder.fit(df2).transform(df2)
df3.show()

+---+--------+-----+--------------+--------------+
| ID|sucursal|venta|indiceSucursal|onehotSucursal|
+---+--------+-----+--------------+--------------+
|  0|       A|10000|           0.0| (3,[0],[1.0])|
|  1|       B| 9000|           3.0|     (3,[],[])|
|  2|       C|15000|           1.0| (3,[1],[1.0])|
|  3|       A|14000|           0.0| (3,[0],[1.0])|
|  4|       A|12000|           0.0| (3,[0],[1.0])|
|  5|       C|19000|           1.0| (3,[1],[1.0])|
|  6|       D|11500|           2.0| (3,[2],[1.0])|
|  7|       D| 5000|           2.0| (3,[2],[1.0])|
+---+--------+-----+--------------+--------------+



**Forma 2**: Utilizando un pipeline

In [34]:
from pyspark.ml import Pipeline

In [48]:
# Indexador sin aplicarlo al DataFrame
string_indexer = StringIndexer(inputCol="categoria", outputCol="indiceCategoria")
# OneHotEncoder sin aplicarlo al Dataframe
one_hot_encoder = OneHotEncoder(inputCol="indiceCategoria", outputCol="onehotCategoria")

# Pipeline con las etapas
pipeline = Pipeline(stages=[string_indexer,
                            one_hot_encoder])

# Obtener las asociaciones usando el pipeline
df2 = pipeline.fit(df)

# Transformar el DataFrame
df3 = df2.transform(df)

# Resultado
df3.show()

+---+---------+---------------+---------------+
| ID|categoria|indiceCategoria|onehotCategoria|
+---+---------+---------------+---------------+
|  0|        a|            0.0|  (2,[0],[1.0])|
|  1|        b|            2.0|      (2,[],[])|
|  2|        c|            1.0|  (2,[1],[1.0])|
|  3|        a|            0.0|  (2,[0],[1.0])|
|  4|        a|            0.0|  (2,[0],[1.0])|
|  5|        c|            1.0|  (2,[1],[1.0])|
+---+---------+---------------+---------------+



## 3. Generación de un vector columna (combinando otras columnas)

`VectorAssembler` combina un conjunto de columnas en un solo vector columna. Es útil para combinar atributos originales con aquellos generados por diferentes transformaciones aplicadas en PySpark. Esto es necesario para tener el formato que los modelos de ML de Spark utilizan.

`VectorAssembler` acepta los siguientes tipos de columnas: todos los tipos numéricos, tipos Booleanos, tipos vector. En cada fila, los valores de las columnas de entrada serán concatenados en un vector de un orden especificado.

Ejemplo: Si se tiene un DataFrame con las columnas id, campo1, campo2, campos3, y valor:

     id | campo1 | campo2 |   campos3   | valor
    ----|--------|--------|-------------|------
    204 |   18   |   1.0  | [3, 10, 20] |  5.9

donde `campos3` es una columna de vectores que contiene tres atributos. Se desea combinar `campo1`, `campo2` y `campos3` en un solo vector de atributos llamado `vatributos` para ser usado como predictor de `valor`. Si se indica que las columnas de entrada de `VectorAssembler` son `campo1`, `campo2` y `campos3`, y que la columna de salida es `valor`, luego de la transformación se obtendrá lo siguiente:

     id | campo1 | campo2 |   campos3   | valor |     vatributos
    ----|--------|--------|-------------|-------|----------------------
    204 |   18   |   1.0  | [3, 10, 20] |  5.9  | [18, 1.0, 3, 10, 20]

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

In [ ]:
# Vector denso en Spark
vector = Vectors.dense([3, 10, 20])
vector

In [ ]:
# Creación de un data frame (de una fila)
df = spark.createDataFrame([(204, 18, 1.0, vector, 5.9),
                            (205, 25, 3.5, vector, 6.7)],
                           ["id", "campo1", "campo2", "campos3", "valor"])
df.show()

In [ ]:
# Objeto que juntará columnas para crear una sola columna
assembler = VectorAssembler(inputCols=["campo1", "campo2", "campos3"],
                            outputCol="vatributos")

# Transformar los datos según la columna creada
df2 = assembler.transform(df)
df2.show()

In [ ]:
df2.show(truncate=False)

In [ ]:
# Seleccionar solo las columnas vatributos y valor (usual como entrada a algoritmos supervizados)
df2.select("vatributos", "valor").show()